# ESRGAN 推理脚本 (Inference)

本 Notebook 用于加载训练好的 ESRGAN 模型（如 `ESRGAN_4x_finetune_best.pth`），对低分辨率 (LR) 图像进行超分辨率重建，并与原始高分辨率 (HR) 图像进行对比。支持通过 `UPSCALE_MODE` 在 2x/4x 配置间切换。

In [4]:
import os
import sys
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import random

# 将本地的 APISR_tools 目录添加到系统路径
apisr_tools_path = os.path.abspath('APISR_tools')
if apisr_tools_path not in sys.path:
    sys.path.append(apisr_tools_path)

# 导入 RRDBNet 架构
from architecture.rrdb import RRDBNet

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

PyTorch Version: 2.9.0+cu130
CUDA Available: True


In [5]:
# ==========================================
# 1. 配置参数
# ==========================================

# 推理模式: '2x' 或 '4x'
UPSCALE_MODE = '2x'

# 与训练脚本保持一致的保存前缀
CHECKPOINT_NAME = f'ESRGAN_{UPSCALE_MODE}_finetune'

# 放大倍数 (必须与训练时一致)
SCALE = 2 if UPSCALE_MODE == '2x' else 4

# RRDB block 数量（必须与训练时一致）
# 经验映射：2x 常用 6 blocks，4x 常用 23 blocks
AUTO_NUM_BLOCK = 6 if UPSCALE_MODE == '2x' else 23
MANUAL_NUM_BLOCK = None  # 可手动设置为 6 或 23；None 表示使用自动映射
NUM_BLOCK = MANUAL_NUM_BLOCK if MANUAL_NUM_BLOCK is not None else AUTO_NUM_BLOCK

# 模型路径 (优先加载 best)
MODEL_PATH = f'saved_models/{CHECKPOINT_NAME}_best.pth'

# 测试图像路径
LR_DIR = f'dataset/lowres_{UPSCALE_MODE}/original'
HR_DIR = 'dataset/highres/original'
OUTPUT_DIR = f'results/ESRGAN_{UPSCALE_MODE}_inference'

# 设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"推理模式: {UPSCALE_MODE}, SCALE={SCALE}, NUM_BLOCK={NUM_BLOCK}")
print(f"模型路径: {MODEL_PATH}")
print(f"LR 路径: {LR_DIR}")

Using device: cuda
推理模式: 2x, SCALE=2, NUM_BLOCK=6
模型路径: saved_models/ESRGAN_2x_finetune_best.pth
LR 路径: dataset/lowres_2x/original


In [6]:
# ==========================================
# 2. 加载模型
# ==========================================

# 实例化生成器 (RRDBNet)
# 注意：scale 和 num_block 必须与训练时完全一致
model = RRDBNet(3, 3, scale=SCALE, num_block=NUM_BLOCK).to(device)

# 加载权重
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"未找到模型权重: {MODEL_PATH}")

print(f"正在加载模型权重: {MODEL_PATH}")
checkpoint = torch.load(MODEL_PATH, map_location=device)

# 兼容不同格式的权重文件
if 'params_ema' in checkpoint:
    model.load_state_dict(checkpoint['params_ema'], strict=True)
    print("成功加载 params_ema 权重。")
elif 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'], strict=True)
    print(f"成功加载 Epoch {checkpoint.get('epoch', 'Unknown')} 的权重。")
else:
    model.load_state_dict(checkpoint, strict=True)
    print("成功加载纯 state_dict 权重。")

# 设置为评估模式
model.eval()

正在加载模型权重: saved_models/ESRGAN_2x_finetune_best.pth
成功加载 Epoch 31 的权重。


RRDBNet(
  (conv_first): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (body): Sequential(
    (0): RRDB(
      (rdb1): ResidualDenseBlock(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (rdb2): ResidualDenseBlock(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1),

In [7]:
# ==========================================
# 3. 推理与可视化函数 (分块推理防爆显存)
# ==========================================

def tensor2img(tensor):
    """将 PyTorch Tensor 转换为 numpy 图像 (RGB, 0-255)"""
    img = tensor.squeeze(0).cpu().numpy()
    img = np.transpose(img, (1, 2, 0)) # CHW -> HWC
    img = np.clip(img * 255.0, 0, 255).astype(np.uint8)
    return img

def infer_and_save(lr_path, hr_path=None, save_dir=None, show=True):
    """对单张图像进行推理，保存并可选择显示对比图"""
    # 1. 读取 LR 图像
    lr_img = cv2.imread(lr_path)
    if lr_img is None:
        print(f"无法读取图像: {lr_path}")
        return None
    lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)
    
    # 2. 预处理 (转为 Tensor, 归一化到 [0, 1], 增加 Batch 维度)
    # 对 2x RRDB(scale=2) 来说，网络内部会 pixel_unshuffle(scale=2)，
    # 分块推理时每个块的高宽都需要能被 2 整除。这里将输入 pad 到 4 的倍数，避免块内断言报错。
    orig_h, orig_w = lr_img.shape[:2]
    pad_h, pad_w = 0, 0
    lr_img_model = lr_img
    if SCALE == 2:
        pad_h = (4 - orig_h % 4) % 4
        pad_w = (4 - orig_w % 4) % 4
        if pad_h > 0 or pad_w > 0:
            lr_img_model = cv2.copyMakeBorder(
                lr_img, 0, pad_h, 0, pad_w, borderType=cv2.BORDER_REFLECT_101
            )

    lr_tensor = torch.from_numpy(lr_img_model.transpose(2, 0, 1)).float() / 255.0
    lr_tensor = lr_tensor.unsqueeze(0).to(device)
    
    # 3. 模型推理 (使用分块推理防止 OOM)
    with torch.no_grad():
        with torch.amp.autocast('cuda', enabled=(device.type == 'cuda')): # 开启混合精度加速推理
            # 如果图像太大，直接推理会爆显存。这里使用简单的分块策略
            # 假设 16GB 显存，LR 图像超过 512x512 可能会有风险
            _, _, h, w = lr_tensor.shape
            
            if h * w > 512 * 512:
                # 简单的 2x2 分块推理 (如果图像更大，可以增加分块数量)
                h_half, w_half = h // 2, w // 2
                
                # 预分配输出 Tensor (注意尺寸要乘以 SCALE)
                sr_tensor = torch.zeros((1, 3, h * SCALE, w * SCALE), device=device)
                
                # 左上
                sr_tensor[:, :, :h_half*SCALE, :w_half*SCALE] = model(lr_tensor[:, :, :h_half, :w_half])
                # 右上
                sr_tensor[:, :, :h_half*SCALE, w_half*SCALE:] = model(lr_tensor[:, :, :h_half, w_half:])
                # 左下
                sr_tensor[:, :, h_half*SCALE:, :w_half*SCALE] = model(lr_tensor[:, :, h_half:, :w_half])
                # 右下
                sr_tensor[:, :, h_half*SCALE:, w_half*SCALE:] = model(lr_tensor[:, :, h_half:, w_half:])
            else:
                # 图像较小，直接推理
                sr_tensor = model(lr_tensor)
            
    # 4. 后处理
    sr_img = tensor2img(sr_tensor)

    # 如果做过 padding，裁掉对应的 SR 边界，恢复到原图对应尺寸
    if pad_h > 0 or pad_w > 0:
        sr_img = sr_img[:orig_h * SCALE, :orig_w * SCALE, :]
    
    # 5. 保存图像
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        filename = os.path.basename(lr_path)
        save_path = os.path.join(save_dir, filename)
        # OpenCV 保存需要 BGR 格式
        cv2.imwrite(save_path, cv2.cvtColor(sr_img, cv2.COLOR_RGB2BGR))
    
    # 6. 可视化
    if show:
        if hr_path and os.path.exists(hr_path):
            # 如果有 HR 图像，显示三张图对比
            hr_img = cv2.imread(hr_path)
            hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
            
            # 为了公平对比，将 LR 图像使用双三次插值放大到相同尺寸
            h, w, _ = hr_img.shape
            lr_resized = cv2.resize(lr_img, (w, h), interpolation=cv2.INTER_CUBIC)
            
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            axes[0].imshow(lr_resized)
            axes[0].set_title('LR (Bicubic Upscaled)')
            axes[0].axis('off')
            
            axes[1].imshow(sr_img)
            axes[1].set_title('SR (ESRGAN Output)')
            axes[1].axis('off')
            
            axes[2].imshow(hr_img)
            axes[2].set_title('HR (Ground Truth)')
            axes[2].axis('off')
            
        else:
            # 如果没有 HR 图像，只显示 LR 和 SR
            fig, axes = plt.subplots(1, 2, figsize=(12, 6))
            axes[0].imshow(lr_img)
            axes[0].set_title('LR Input')
            axes[0].axis('off')
            
            axes[1].imshow(sr_img)
            axes[1].set_title('SR (ESRGAN Output)')
            axes[1].axis('off')
            
        plt.tight_layout()
        plt.show()
        
    return sr_img

In [8]:
# ==========================================
# 4. 批量推理并保存
# ==========================================
from tqdm import tqdm

# 获取所有 LR 图像路径
lr_paths = sorted(glob.glob(os.path.join(LR_DIR, '*.*')))
hr_paths = sorted(glob.glob(os.path.join(HR_DIR, '*.*')))

if not lr_paths:
    print("未找到测试图像，请检查路径。")
else:
    print(f"开始批量推理，结果将保存至: {OUTPUT_DIR}")
    # 随机抽取 3 张图像进行可视化展示
    num_samples = min(3, len(lr_paths))
    sample_indices = random.sample(range(len(lr_paths)), num_samples)
    
    for idx, lr_path in enumerate(tqdm(lr_paths, desc="Inference Progress")):
        filename = os.path.basename(lr_path)
        hr_path = os.path.join(HR_DIR, filename)
        
        # 只有被抽中的 3 张图片会显示可视化对比，其他的只保存
        show_vis = (idx in sample_indices)
        if show_vis:
            print(f"\nVisualizing: {filename}")
            
        infer_and_save(lr_path, hr_path=hr_path, save_dir=OUTPUT_DIR, show=show_vis)
        
    print("批量推理完成！")

开始批量推理，结果将保存至: results/ESRGAN_2x_inference


Inference Progress:   0%|          | 0/434 [00:00<?, ?it/s]


AssertionError: 

In [ ]:
# ==========================================
# 5. 图像质量评估 (NIQE, MANIQA, CLIPIQA)
# ==========================================
# 注意：运行此代码块需要安装 pyiqa 库
# 可以通过取消注释下一行来安装：
# !pip install pyiqa
from tqdm import tqdm
try:
    import pyiqa
except ImportError:
    print("未检测到 pyiqa 库，准备退出。")
    exit()


print("正在初始化评估指标 (首次运行会自动下载预训练权重)...")
# 初始化无参考图像质量评估指标 (No-Reference IQA)
niqe_metric = pyiqa.create_metric('niqe', device=device)
maniqa_metric = pyiqa.create_metric('maniqa', device=device)
clipiqa_metric = pyiqa.create_metric('clipiqa', device=device)

sr_paths = sorted(glob.glob(os.path.join(OUTPUT_DIR, '*.*')))

niqe_scores = []
maniqa_scores = []
clipiqa_scores = []

print("开始评估生成的 SR 图像...")
for sr_path in tqdm(sr_paths, desc="Evaluating"):
    # pyiqa 可以直接接受图像路径进行评估
    niqe_score = niqe_metric(sr_path).item()
    maniqa_score = maniqa_metric(sr_path).item()
    clipiqa_score = clipiqa_metric(sr_path).item()
    
    niqe_scores.append(niqe_score)
    maniqa_scores.append(maniqa_score)
    clipiqa_scores.append(clipiqa_score)

print("\n" + "="*40)
print("评估结果 (Evaluation Results):")
print("="*40)
print(f"Average NIQE   : {np.mean(niqe_scores):.4f} (越低越好, Lower is better)")
print(f"Average MANIQA : {np.mean(maniqa_scores):.4f} (越高越好, Higher is better)")
print(f"Average CLIPIQA: {np.mean(clipiqa_scores):.4f} (越高越好, Higher is better)")
print("="*40)

正在初始化评估指标 (首次运行会自动下载预训练权重)...
Loading pretrained model MANIQA from C:\Users\admin\.cache\torch\hub\pyiqa\ckpt_koniq10k.pt
开始评估生成的 SR 图像...


Evaluating: 100%|██████████| 434/434 [16:40<00:00,  2.31s/it]  


评估结果 (Evaluation Results):
Average NIQE   : 5.2683 (越低越好, Lower is better)
Average MANIQA : 0.2639 (越高越好, Higher is better)
Average CLIPIQA: 0.4848 (越高越好, Higher is better)
